In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.max_columns = None

from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer

from sklearn import preprocessing
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS, SelectFromModel
from xgboost import XGBClassifier as XGBC
from xgboost import XGBRegressor as XGBR

In [2]:

train5_cat0= pd.read_csv('Timestep5_train_cat0.csv')
train5_cat1= pd.read_csv('Timestep5_train_cat1.csv')
test5_cat0= pd.read_csv('Timestep5_test_cat0.csv')
test5_cat1= pd.read_csv('Timestep5_test_cat1.csv')

traintest= pd.read_csv('Mmerged_icu_mean.csv')
hold= pd.read_csv('MmergedHoldout.csv')

In [3]:
full=pd.concat([traintest, hold])

full['id'] = full['id'].astype(str)

In [4]:
#get id
train0ID= train5_cat0['id'].astype(str)
test0ID= test5_cat0['id'].astype(str)
train1ID= train5_cat1['id'].astype(str)
test1ID= test5_cat1['id'].astype(str)

In [5]:
#split with same id

#cat0
train0= full.loc[full['id'].isin(train0ID)]
test0= full.loc[full['id'].isin(test0ID)]

#cat1
train1= full.loc[full['id'].isin(train1ID)]
test1= full.loc[full['id'].isin(test1ID)]

In [6]:
# train0.to_csv('train0_8-2split.csv', index=False)
# train1.to_csv('train1_8-2split.csv', index=False)
# test0.to_csv('test0_8-2split.csv', index=False)
# test1.to_csv('test1_8-2split.csv', index=False)

In [150]:
print(train_cat0.shape)
print(train_cat1.shape)
print(test_cat0.shape)
print(test_cat1.shape)

(7136, 67)
(4766, 67)
(1783, 67)
(1193, 67)


In [151]:
train_cat0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7136 entries, 6 to 2035
Data columns (total 67 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id                                                               7136 non-null   object 
 1   hosp_admittime                                                   7136 non-null   object 
 2   hosp_dischtime                                                   7136 non-null   object 
 3   icu_intime                                                       7136 non-null   object 
 4   icu_outtime                                                      7136 non-null   object 
 5   los_icu                                                          7136 non-null   float64
 6   icu_death                                                        7136 non-null   int64  
 7   gender                                    

In [152]:
regX_train=train.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome', 'icu_death', 'los_icu'], axis=1)
regX_test=test.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome', 'icu_death', 'los_icu'],axis=1)

regy_train=train['los_icu']
regy_test=test['los_icu']

In [153]:
#normalise
num_cols = regX_train.columns[regX_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_train[num_cols] = scaler.fit_transform(regX_train[num_cols])

num_cols = regX_test.columns[regX_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_test[num_cols] = scaler.fit_transform(regX_test[num_cols])


# Random forest

In [92]:
#select frm model

sel = SelectFromModel(RandomForestRegressor())
sel.fit(regX_train, regy_train)

selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['admission_age' 'weight_admit' 'height' 'charlson_score' 'aniongap'
 'bicarbonate' 'bun' 'calcium' 'chloride' 'creatinine' 'glucose' 'sodium'
 'potassium' 'hematocrit' 'hemoglobin' 'mch' 'mchc' 'mcv' 'platelet' 'rbc'
 'rdw' 'wbc' 'inr' 'pt' 'ptt']
25


In [93]:
rfr = RandomForestRegressor()

parameters = {'max_depth':np.arange(start=1,stop=10,step=1),
             'n_estimators':np.arange(start=100,stop=500,step=100),
             }
KfoldCV = KFold(n_splits=5, shuffle =True, random_state=1)
Best_rfr = GridSearchCV(
    rfr,
    param_grid=parameters,
    cv=KfoldCV,
)

Best_rfr.fit(regX_train,regy_train)


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=RandomForestRegressor(),
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': array([100, 200, 300, 400])})

In [94]:
Best_rfr.best_estimator_

RandomForestRegressor(max_depth=6, n_estimators=300)

In [118]:
Best_rfr.fit(regX_train[selected_feats],regy_train)

KeyboardInterrupt: 

In [ ]:
rfr_pred = Best_rfr.predict(regX_test[selected_feats])
rfr_MSE = mean_squared_error(regy_test, rfr_pred)

rfr_MSE

# XGBoost

In [96]:
# xgbr =XGBR()

# xgbr.fit(regX_train,regy_train)

In [97]:
# thresholds = np.sort(xgbr.feature_importances_)

In [98]:
# thresholds

In [99]:
# for thresh in thresholds:
#     selection = SelectFromModel(xgbr, threshold=thresh, prefit=True)
#     select_regX_train = selection.transform(regX_train)
#     selection_model = xgbr()
#     selection_model.fit(select_regX_train, regy_train)
#     select_regX_test = selection.transform(regX_test)
#     predictions = selection_model.predict(select_regX_test)
#     accuracy = accuracy_score(regy_test, predictions)
#     print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_regX_train.shape[1], accuracy*100.0))

In [100]:
# selected=pd.DataFrame()
# selected['col']=regX_train.columns
# selected['rank']=xgbr.feature_importances_
# selected.sort_values(by='rank', ascending=False,inplace=True)

# top6=selected[:6]
# selected_feats=top6['col'].to_list()
# selected_feats

In [101]:

xgbr = XGBR()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
kfoldCV = KFold(n_splits=5)
Best_xgbr = GridSearchCV(
    xgbr,
    param_grid=parameters,
    cv=kfoldCV,
)

Best_xgbr.fit(regX_train,regy_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])})

In [102]:
Best_xgbr.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.35000000000000003,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=14, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [103]:
xgbr_pred = Best_xgbr.predict(regX_test)
xgbr_MSE = mean_squared_error(regy_test, xgbr_pred)

xgbr_MSE

0.4675993805791646

# Ridge

In [127]:
from sklearn.linear_model import Ridge, RidgeCV


In [128]:
l2=Ridge()
l2.fit(regX_train, regy_train)

Ridge()

In [129]:
sel = SelectFromModel(l2)
sel.fit(regX_train, regy_train)

selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['aniongap' 'bicarbonate' 'chloride' 'sodium' 'hematocrit' 'hemoglobin'
 'mch' 'mchc' 'mcv' 'rbc' 'inr' 'pt']
12


In [130]:
Best_l2 = RidgeCV(alphas=np.logspace(-6, 6, 13))

Best_l2.fit(regX_train[selected_feats],regy_train)

RidgeCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]))

In [131]:
Best_l2.alpha_

10.0

In [132]:
l2_pred = Best_l2.predict(regX_test[selected_feats])
l2_MSE = mean_squared_error(regy_test, l2_pred)

l2_MSE

0.18376470715870666

# Support vector

In [155]:
from sklearn.svm import SVR


In [156]:
svr = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr.fit(regX_train, regy_train)

SVR(C=100, gamma=0.1)

In [157]:
svr = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)

# parameters = {
#     'C':np.arange(start=50,stop=150,step=50),
#     'gamma':np.arange(start=0.1,stop=1,step=0.1),
# }
# kfoldCV = KFold(n_splits=5)
# Best_svr = GridSearchCV(
#     svr,
#     param_grid=parameters,
#     cv=kfoldCV,
# )

Best_svr.fit(regX_train,regy_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVR(C=100, gamma=0.1),
             param_grid={'C': array([ 50, 100]),
                         'gamma': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])})

In [158]:
Best_svr.best_estimator_

SVR(C=50, gamma=0.30000000000000004)

In [159]:
svr_pred = Best_svr.predict(regX_test)
svr_MSE = mean_squared_error(regy_test, svr_pred)

svr_MSE

0.47008712988947843